<a href="https://colab.research.google.com/github/shankervalipireddyai/RAG-Graph-LLM/blob/master/freshprompt_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [133]:
#@title Installing required Python packages


!pip install openai
!pip install google-search-results
!pip install requests
!pip install gql


In [213]:
#@title Importing Python libraries and modules


import os
import re
import time
import datetime
import pytz
import dateutil
import requests
import json
import csv

import pandas as pd

from google.colab import files
import ipywidgets as widgets
from IPython.display import display

from openai import OpenAI
import tabulate
import textwrap

from serpapi import GoogleSearch

current_date = datetime.datetime.now(
        pytz.timezone("America/Los_Angeles")
    ).strftime("%B %d, %Y")

In [214]:
from openai import OpenAI

# API keys initialization and assertion

# OpenAI's API key (sign up at https://platform.openai.com/signup to get $5 in
# free credit that can be used during your first 3 months)
openai_api_key = "sk-proj-UHOtWEkOuyPYTgN4gARFT3BlbkFJpqdnjQxXi8p55lvyupqW"  # @param {type:"string"}
openai_client = OpenAI(
    api_key=openai_api_key,
)

# SerpApi's API key (sign up at https://serpapi.com/users/sign_up?plan=free for
# a free plan with 100 searches/month)
serpapi_api_key = ""  # @param {type:"string"}

# MediaTech News API key (obtain your key from the MediaTech News API signup)
mediathech_api_key = "9d07b5b2291388a14541b99f11f0b9a5"  # @param {type:"string"}

# Assert that API keys are set
assert openai_api_key, "OpenAI's API key is not set"
#assert serpapi_api_key, "SerpApi's API key is not set"
assert mediathech_api_key, "MediaTech News API key is not set"

# Now, you can use these API keys in your requests or API calls accordingly.



In [215]:
#@title Function calling for the base LLM


def call_llm_api(prompt, model, temperature, max_tokens, chat_completions=True):
  # See https://platform.openai.com/docs/guides/gpt for details
  if chat_completions:
    # Chat completions API
    response = openai_client.chat.completions.create(
        model=model,
        temperature=temperature,
        max_tokens=max_tokens,
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a helpful assistant. Answer as concisely as"
                    f" possible. Knowledge cutoff: {current_date}."
                ),
            },
            {"role": "user", "content": "What's today's date?"},
            {
                "role": "assistant",
                "content": f"Today is {current_date} in Pacific Standard Time.",
            },
            {"role": "user", "content": prompt},
        ],
    )
    return response.choices[0].message.content

  else:
    # Completions API
    response = openai_client.completions.create(
        model=model,
        temperature=temperature,
        max_tokens=max_tokens,
        prompt=prompt,
    )
    return response.choices[0].text


In [216]:
#@title Function calling for the search engine


def call_search_engine(query):
  params = {
    "q": query,
    # "location": "California, United States",
    "hl": "en",
    "gl": "us",
    "google_domain": "google.com",
    "api_key": serpapi_api_key,

  }

  search = GoogleSearch(params)
  return search.get_dict()


import requests

def call_news_api(query, api_key):
    """
    Fetch news articles from the MediaTech News API based on the provided query.

    Args:
        query (str): Keyword or topic to fetch news about.
        api_key (str): MediaTech News API key.

    Returns:
        dict: A dictionary containing news articles and additional information.
    """
    url = "http://api.mediastack.com/v1/news"  # Update with the correct URL
    params = {
        "access_key": api_key,
        "keywords": query,
        "sort": "published_desc",  # Sorting articles by most recent
        "limit": 10  # Adjust the number of results as needed
    }
    response = requests.get(url, params=params)

    if response.status_code == 200:
        return response.json()['data']  # Returns the JSON response as a dictionary
    else:
        return {"error": "Failed to fetch news", "status_code": response.status_code}

# Example usage
api_key = "9d07b5b2291388a14541b99f11f0b9a5"  # Replace with your actual API key
query = "latest technology news"
news_data = call_news_api(query, api_key)
print(news_data)


[{'author': 'India Post Newspaper', 'title': 'Suspense continues over Rahul, Priyanka’s candidatures from Raebareli and Amethi; final decision today', 'description': 'NEW DELHI: With just a day left for the filing of nominations for Phase 5 of the Lok Sabha elections, the suspense continued over the Congress&#8217; candidates for the Amethi and Raebareli constituencies of Uttar Pradesh, once considered a stronghold of the Nehru-Gandhi family. According to sources, Congress leader Rahul Gandhi, who is already on [&#8230;]', 'url': 'https://indiapost.com/suspense-continues-over-rahul-priyankas-candidatures-from-raebareli-and-amethi-final-decision-today/', 'source': 'indiapost', 'image': None, 'category': 'general', 'language': 'en', 'country': 'us', 'published_at': '2024-05-02T06:36:57+00:00'}, {'author': 'NewsDesk', 'title': 'Later this month, Pakistan will reveal their T20 World Cup team: report', 'description': 'LAHORE: Because of certain players&#8217; performance and fitness concern

In [217]:
#@title Utility functions for FreshPrompt
from dateutil import parser

def is_date(string, fuzzy=False):
  # Parse a string into a date and check its validity
  try:
      dateutil.parser.parse(string, fuzzy=fuzzy)
      return True
  except ValueError:
      return False


def format_date1(d):
    """
    Converts a date input to a more readable string format. Handles different types of date inputs.

    Args:
        d (str, float, None): Date input which can be ISO format string, timestamp, or None.

    Returns:
        str: Formatted date string or an error message if the input is invalid.
    """
    if d is None:
        return "Date not available"

    # If the input is a float (potentially a timestamp), try converting it to a datetime object
    if isinstance(d, (float, int)):
        try:
            return datetime.datetime.fromtimestamp(d).strftime('%b %d, %Y')
        except Exception as e:
            return f"Error converting timestamp: {e}"

    # If the input is a string, try parsing it as a date
    if isinstance(d, str):
        try:
            # Check for relative times described in the string
            for t in ["second", "minute", "hour"]:
                if f"{t} ago" in d or f"{t}s ago" in d:
                    # Assume current time minus the relative time mentioned
                    current_date = datetime.datetime.now()
                    if 'second' in t:
                        delta = datetime.timedelta(seconds=int(re.findall(r'\d+', d)[0]))
                    elif 'minute' in t:
                        delta = datetime.timedelta(minutes=int(re.findall(r'\d+', d)[0]))
                    elif 'hour' in t:
                        delta = datetime.timedelta(hours=int(re.findall(r'\d+', d)[0]))
                    return (current_date - delta).strftime('%b %d, %Y')

            # Regular date parsing
            return dateutil.parser.parse(d).strftime('%b %d, %Y')
        except ValueError:
            return "Invalid date string"

    return "Unsupported date format"


def extract_source_webpage(link):
  # Extract source webpage
  return (
      link.strip()
      .replace("https://www.", "")
      .replace("http://www.", "")
      .replace("https://", "")
      .replace("http://", "")
      .split("/")[0]
  )

def extract_domain(url):
    # Extract the domain from a URL.
    return re.sub(r'https?://(www\.)?', '', url).split('/')[0]

def simplify_displayed_link(displayed_link):
  # Simplify displayed link
  if displayed_link is None:
    return None
  return extract_source_webpage(displayed_link.split(' › ')[0])



def format_news_article(article):

    """ Format a single news article data for clearer presentation and usage. """
    # Ensure the input is a dictionary
    #print(article)
    if not isinstance(article, dict):
        return {"error": "The input must be a dictionary."}

    # Check if the dictionary is empty
    if not article:
        return {"error": "The input dictionary cannot be empty."}

    # Define required fields
    required_fields = ['author', 'title', 'description', 'url', 'source', 'published_at']

    # Check for missing required fields
    missing_fields = [field for field in required_fields if field not in article or not article[field]]
    if missing_fields:
        return {"error": f"Missing required fields: {', '.join(missing_fields)}"}

    try:
        # Extract and format data
        formatted_article = {
            'author': article.get('author', 'Author not specified'),
            'title': article.get('title', 'No title provided'),
            'description': article.get('description', 'No description provided'),
            'url': article.get('url', 'No URL provided'),
            'source': article.get('source', 'Source not specified'),
            'image': article.get('image', 'No image available'),
            'category': article.get('category', 'General'),
            'language': article.get('language', 'EN'),
            'country': article.get('country', 'US'),
            'published_at': (article['published_at'])
        }
        return formatted_article
    except Exception as e:
        return {"error": f"An error occurred while processing the article: {str(e)}"}

q = "latest news in tracy"
articles = call_news_api(q, api_key)
formatted_articles = [format_news_article(article) for article in articles]
#print(formatted_articles)

def format_date(d):
  # Standardize the date format for each search result
  date = dateutil.parser.parse(current_date, fuzzy=True).strftime("%b %d, %Y")
  if d is None:
    return None

  for t in ["second", "minute", "hour"]:
    if f"{t} ago" in d or f"{t}s ago" in d:
      return date

  t = "day"
  if f"{t} ago" in d or f"{t}s ago" in d:
    n_days = int(re.search("(\d+) days? ago", d).group(1))
    return (
        datetime.datetime.strptime(date, "%b %d, %Y")
        - datetime.timedelta(days=n_days)
    ).strftime("%b %d, %Y")

  try:
    return dateutil.parser.parse(d, fuzzy=True).strftime("%b %d, %Y")
  except ValueError:
    for x in d.split():
      if is_date(x):
        return dateutil.parser.parse(x, fuzzy=True).strftime("%b %d, %Y")




def format_search_results(search_data, title_field=None, highlight_field=None):

  # Standardize search results as shown in Figure 3 (left) in the paper
  field = 'snippet_highlighted_words'
  if field in search_data and isinstance(search_data[field], list):
    search_data[field] = ' | '.join(search_data[field])

  field = 'displayed_link'
  if field in search_data:
    search_data[field] = simplify_displayed_link(search_data[field])
  # edge case 1
  if search_data.get('type') == 'local_time':
    source = search_data.get('displayed_link')
    date = format_date(search_data.get('date'))
    title = search_data.get('title')

    snippet = search_data.get('snippet')
    if snippet is None and 'result' in search_data:
      if 'extensions' in search_data and isinstance(
          search_data['extensions'], list
      ):
        snippet = '\n\t'.join(
            [search_data['result']] + search_data['extensions']
        )
      else:
        snippet = search_data['result']

    highlight = search_data.get('snippet_highlighted_words')
    if highlight is None and 'result' in search_data:
      highlight = search_data['result']

  # edge case 2
  elif 'type' in search_data and search_data['type'] == 'population_result':
    source = search_data.get('displayed_link')
    if source is None and 'sources' in search_data:
      if (
          isinstance(search_data['sources'], list)
          and 'link' in search_data['sources'][0]
      ):
        source = extract_source_webpage(search_data['sources'][0]['link'])

    date = format_date(search_data.get('date'))
    if date is None and 'year' in search_data:
      date = format_date(search_data['year'])

    title = search_data.get('title')

    snippet = search_data.get('snippet')
    if snippet is None and 'population' in search_data:
      if 'place' in search_data:
        snippet = '\n\t'.join(
            [
                f"{search_data['place']} / Population",
            ]
            + [
                search_data['population'],
            ]
        )
      else:
        snippet = search_data['population']

    highlight = search_data.get('snippet_highlighted_words')
    if highlight is None and 'population' in search_data:
      highlight = search_data['population']

  else:
    source = search_data.get('displayed_link')
    date = format_date(search_data.get('date'))
    title = (
        search_data.get('title')
        if title_field is None
        else search_data.get(title_field)
    )
    highlight = (
        search_data.get('snippet_highlighted_words')
        if highlight_field is None
        else search_data.get(highlight_field)
    )
    snippet = search_data.get('snippet', '')

    if 'rich_snippet' in search_data:
      for key in ['top', 'bottom']:
        if (
            key in search_data['rich_snippet']
            and 'extensions' in search_data['rich_snippet'][key]
        ):
          snippet = '\n\t'.join(
              [snippet] + search_data['rich_snippet'][key]['extensions']
          )

    if 'list' in search_data:
      assert isinstance(search_data['list'], list)
      snippet = '\n\t'.join([snippet] + search_data['list'])

    if 'contents' in search_data and 'table' in search_data['contents']:
      tbl = search_data['contents']['table']
      assert isinstance(tbl, list)
      snippet += '\n'
      for row in tbl:
        snippet += f'\n{",".join(row)}'

    if snippet is not None and snippet.strip() == '':
      snippet = None

  return {
      'source': source,
      'date': date,
      'title': title,
      'snippet': snippet,
      'highlight': highlight,
  }


def format_knowledge_graph(search_data):
  # Standardize knowledge graphs as shown in Figure 3 (left) in the paper
  source = None
  if "source" in search_data and "link" in search_data["source"]:
    source = extract_source_webpage(search_data["source"]["link"])

  date = None

  title = None
  if "title" in search_data:
    title = search_data["title"]
    if "type" in search_data:
      title += f"\n\t{search_data['type']}"

  snippet = ""
  for field in search_data:
    if (
        (field not in ["title", "type", "kgmid"])
        and ("_link" not in field)
        and ("_stick" not in field)
        and isinstance(search_data[field], str)
        and not search_data[field].startswith("http")
    ):
      snippet += f"\n\t{field}: {search_data[field]}"

  if snippet.strip() == "":
    snippet = None
  else:
    snippet = snippet.strip()

  highlight = None

  return {
      "source": source,
      "date": date,
      "title": title,
      "snippet": snippet,
      "highlight": highlight,
  }


def format_questions_and_answers(search_data):
  # Standardize questions and answers as shown in Figure 3 (left) in the paper
  source = None
  if "link" in search_data:
    source = extract_source_webpage(search_data["link"])

  date = None

  title = None
  if "question" in search_data:
    title = search_data["question"]

  snippet = None
  if "answer" in search_data:
    snippet = search_data["answer"]

  highlight = None

  return {
      "source": source,
      "date": date,
      "title": title,
      "snippet": snippet,
      "highlight": highlight,
  }


def freshprompt_format(
    question,
    search_data,
    reasoning_and_answer,
    num_organic_results,
    num_related_questions,
    num_questions_and_answers,
    num_retrieved_evidences,
):
  """Build FreshPrompt for each question

  Args:
    question: The question to process.
    search_data: Search data.
    reasoning_and_answer: The reasoning and answer.
    num_organic_results: Number of organic results to keep.
    num_related_questions: Number of related questions to keep.
    num_questions_and_answers: Number of questions and answers to keep.
    num_retrieved_evidences: Number of retrieved evidences to keep.

  Returns:
    A prompt that incorporates retrieved evidences for each question.
  """


  df = pd.DataFrame(columns=['source', 'date', 'title', 'snippet', 'highlight'])


  # Organic results
  organic_results = [None] * num_organic_results
  for k in range(num_organic_results):
    if (
        'organic_results' in search_data
        and len(search_data['organic_results']) > k
    ):
      organic_results[k] = format_search_results(
          search_data['organic_results'][k]
      )
    else:
      organic_results[k] = format_search_results({})

  for d in organic_results[::-1]:
    df = pd.concat([df, pd.DataFrame([d])], ignore_index=True)

  # Related questions
  related_questions = [None] * num_related_questions
  for k in range(num_related_questions):
    if (
        'related_questions' in search_data
        and len(search_data['related_questions']) > k
    ):
      related_questions[k] = format_search_results(
          search_data['related_questions'][k], title_field='question'
      )
    else:
      related_questions[k] = format_search_results({})

  for d in related_questions[::-1]:
    df = pd.concat([df, pd.DataFrame([d])], ignore_index=True)

  # Questions and Answers
  questions_and_answers = [None] * num_questions_and_answers
  for k in range(num_questions_and_answers):
    if (
        'questions_and_answers' in search_data
        and len(search_data['questions_and_answers']) > k
    ):
      questions_and_answers[k] = format_questions_and_answers(
          search_data['questions_and_answers'][k]
      )
    else:
      questions_and_answers[k] = format_questions_and_answers({})

  for d in questions_and_answers[::-1]:
    df = pd.concat([df, pd.DataFrame([d])], ignore_index=True)

  # Knowledge graph
  knowledge_graph = None
  if 'knowledge_graph' in search_data:
    knowledge_graph = format_knowledge_graph(search_data['knowledge_graph'])
  else:
    knowledge_graph = format_knowledge_graph({})
  df = pd.concat([df, pd.DataFrame([knowledge_graph])], ignore_index=True)

  # Answer box
  answer_box = None
  if 'answer_box' in search_data:
    answer_box = format_news_article(
        search_data['answer_box'], highlight_field='answer'
    )
  else:
    answer_box = format_news_article({})
  df = pd.concat([df, pd.DataFrame([answer_box])], ignore_index=True)

  # Sort by date
  df['date'] = df['date'].apply(lambda x: (x))
  df['datetime'] = pd.to_datetime(df['date'], errors='coerce')
  df = df.sort_values(by='datetime', na_position='first')
  df.replace({pd.NaT: None}, inplace=True)
  df = df.dropna(how='all')

  # Select top_k supporting evidences overall
  evidences = []

  for _, row in df.tail(num_retrieved_evidences).iterrows():
    evidences.append(
        f"""\n\nsource: {row['source']}\ndate: {row['date']}\ntitle: {row['title']}\nsnippet: {row['snippet']}\nhighlight: {row['highlight']}"""
    )

  return (
      ''.join(
          [
              f'\n\n\nquery: {question}',
          ]
          + evidences
      )
      + f'\n\nquestion: {question}{reasoning_and_answer}'
  )


def format_news_article(article, highlight_field=None):
    """Standardize and format news article details."""
    source = article.get('source', 'No source provided')
    date = article.get('published_at', 'No publication date provided')
    title = article.get('title', 'No title provided')
    snippet = article.get('description', 'No description provided')
    highlight = article.get(highlight_field) if highlight_field and highlight_field in article else None
    return {
        'source': source,
        'date': date,
        'title': title,
        'snippet': snippet,
        'highlight': highlight,
    }

def freshprompt_format_news(
    question,
    news_data,
    reasoning_and_answer,
    num_articles,
    num_related_questions,
    num_questions_and_answers,
    num_retrieved_evidences
):
    """Build FreshPrompt for each question specifically for news data."""
    df = pd.DataFrame(columns=['source', 'date', 'title', 'snippet', 'highlight'])

    # Process each article in the news data
    for article in news_data[:num_articles]:
        formatted_article = format_news_article(article)
        df = pd.concat([df, pd.DataFrame([formatted_article])], ignore_index=True)

    # Sort articles by publication date
    df['datetime'] = pd.to_datetime(df['date'], errors='coerce')
    df = df.sort_values(by='datetime', ascending=False)
    df.replace({pd.NaT: None}, inplace=True)
    df = df.dropna(how='all')

    # Generate evidences string for FreshPrompt
    evidences = []
    for _, row in df.iterrows():
        evidences.append(
            f"\n\nsource: {row['source']}\ndate: {row['date']}\ntitle: {row['title']}\nsnippet: {row['snippet']}\nhighlight: {row['highlight']}"
        )

    # Return formatted FreshPrompt
    return (
        ''.join(
            [
                f'\n\n\nquery: {question}',
            ] + evidences
        ) + f'\n\nquestion: {question}{reasoning_and_answer}'
    )


In [218]:
#@title Demonstration examples


demo_news_questions = [
    "What major tech merger was completed in 2024?",
    "Which country hosted the 2024 Summer Olympics?",
    "What was the biggest environmental concern in 2024?",
    "Who won the Nobel Peace Prize in 2024?",
    "Which tech company reached a market value of $2 trillion in 2024?"
]

concise_demo_news_reasonings_and_answers = [
    "The major tech merger completed in 2024 was between Company X and Company Y, enhancing their global market reach.",
    "The 2024 Summer Olympics were hosted by Paris, France.",
    "The biggest environmental concern in 2024 was the rapid melting of the Arctic ice cap.",
    "The Nobel Peace Prize in 2024 was awarded to Dr. Jane Goodall for her lifelong contributions to environmental conservation and peace.",
    "Tech company NeuralTech reached a market value of $2 trillion in 2024, becoming the fastest company to achieve this milestone."
]

verbose_demo_news_reasonings_and_answers = [
    (
        "In 2024, the most significant merger in the tech industry was completed"
        " between Company X and Company Y. This merger was notable for combining"
        " two of the largest players in the AI and cloud computing market, setting"
        " a precedent for future tech industry consolidations."
    ),
    (
        "Paris, France was the host city for the 2024 Summer Olympics. The games"
        " were celebrated for their emphasis on sustainability and for including"
        " new sports such as skateboarding and surfing, reflecting modern trends and"
        " athletic interests."
    ),
    (
        "The most pressing environmental issue in 2024 was the unprecedented rate"
        " of Arctic ice melt, which scientists warned could lead to significant"
        " rises in sea levels worldwide. This phenomenon has sparked global"
        " discussions on more aggressive climate actions."
    ),
    (
        "Dr. Jane Goodall was awarded the Nobel Peace Prize in 2024. She was recognized"
        " for her extensive work in promoting peace through environmental activism"
        " and her efforts in wildlife conservation, particularly with chimpanzees."
    ),
    (
        "In 2024, NeuralTech, a leading tech company known for its innovations in"
        " neural networking and AI technology, reached a market capitalization of"
        " $2 trillion. This achievement marked it as one of the most valuable"
        " companies in the world, achieved in record time since its inception."
    )
]


demo_questions = [
    "What year is considered Albert Einstein's annus mirabilis?",
    "Which photographer took the most expensive photograph in the world?",
    "How many days are left until the 2023 Grammy Awards?",
    "How many years ago did the Boxing Day Tsunami happen?",
    (
        "When did Amazon become the first publicly traded company to exceed a"
        " market value of $3 trillion?"
    ),
]

concise_demo_reasonings_and_answers = [
    (
        "1905 is considered Albert Einstein's annus mirabilis, his miraculous"
        " year."
    ),
    (
        'The most expensive photograph in the world is "Le Violon d\'Ingres".'
        " The photograph was created by Man Ray."
    ),
    (
        "The 2023 Grammy Awards ceremony was held on February 5, 2023. Thus,"
        " the ceremony has already taken place."
    ),
    (
        "The disaster occurred on December 26, 2004. Thus, it happened 19 years"
        " ago."
    ),
    "Amazon's market capitalization has never exceeded $3 trillion.",
]

verbose_demo_reasonings_and_answers = [
    (
        "In the year of 1905, Albert Einstein published four groundbreaking"
        " papers that revolutionized scientific understanding of the universe."
        " Thus, scientists call 1905 Albert Einstein's annus mirabilis — his"
        " year of miracles."
    ),
    (
        "Man Ray's famed \"Le Violon d'Ingres\" became the most expensive"
        " photograph ever to sell at auction, sold for $12.4 million on May"
        " 14th, 2022 at Christie's New York. The black and white image, taken"
        " in 1924 by the American surrealist artist, transforms a woman's naked"
        " body into a violin by overlaying the picture of her back with"
        " f-holes. Thus, Man Ray is the photographer who took the most"
        " expensive photograph in the world."
    ),
    (
        "The 2023 Grammy Awards, officially known as the 65th Annual Grammy"
        " Awards ceremony, was held in Los Angeles on February 5, 2023. Thus,"
        " the event has already taken place."
    ),
    (
        "The Boxing Day Tsunami refers to the 2004 Indian Ocean earthquake and"
        " tsunami, which is one of the deadliest natural disasters in recorded"
        " history, killing an estimated 230,000 people across 14 countries. The"
        " disaster occurred on December 26, 2004, which is 19 years ago."
    ),
    (
        "Amazon's market capitalization hit a peak of roughly $1.9 trillion in"
        " July 2021. In 2022, Amazon became the first public company ever to"
        " lose $1 trillion in market value. Thus, Amazon's market value has"
        " never exceeded $3 trillion. In fact, Apple became the first publicly"
        " traded U.S. company to exceed a market value of $3 trillion in"
        " January 2022."
    ),
]

prefix = (
    f"\nanswer: As of today {current_date}, the most up-to-date and relevant"
    " information regarding this query is as follows. "
)

concise_demo_reasonings_and_answers = [
    prefix + x for x in concise_demo_reasonings_and_answers
]
verbose_demo_reasonings_and_answers = [
    prefix + x for x in verbose_demo_reasonings_and_answers
]



In [219]:
#@title Retrieving search data for demonstration examples


demo_search_data = [call_search_engine(q) for q in demo_questions]

api_key = "9d07b5b2291388a14541b99f11f0b9a5"  # Replace with your actual API key

# List of demo questions or topics to fetch news about
demo_questions = [
    "What year is considered Albert Einstein's annus mirabilis?",
    "Which photographer took the most expensive photograph in the world?",
    "How many days are left until the 2023 Grammy Awards?",
    "How many years ago did the Boxing Day Tsunami happen?",
    "When did Amazon become the first publicly traded company to exceed a market value of $1 trillion?"
]

q = "latest news in San Fransisco CA"

# Fetch news for each question using the fetch_news_data function
demo_search_data = [call_news_api(q, api_key) for question in demo_questions]

print(demo_search_data)


[[{'author': 'Sávio Benevides', 'title': 'Nkechi Blessing e Laide Bakare reagem quando Taiwo Hassan e Yinka Quadri finalmente se reconciliam', 'description': 'A postagem Nkechi Blessing e Laide Bakare reagem quando Taiwo Hassan e Yinka Quadri finalmente se reconciliam apareceu pela primeira vez no Kemi Filani News. As atrizes de Nollywood, Nkechi Blessing e Laide Bakare reagiram após a reconciliação de seus colegas mais antigos, Taiwo Hassan e Yinka Quadri. Kemi Filani relatou anos atrás que o [&#8230;]The post Nkechi Blessing e Laide Bakare reagem quando Taiwo Hassan e Yinka Quadri finalmente se reconciliam appeared first on Jornal Espalha Fato.', 'url': 'https://jornalespalhafato.com/2024/05/02/nkechi-blessing-e-laide-bakare-reagem-quando-taiwo-hassan-e-yinka-quadri-finalmente-se-reconciliam/', 'source': 'jornalespalhafato', 'image': None, 'category': 'entertainment', 'language': 'pt', 'country': 'br', 'published_at': '2024-05-02T06:37:34+00:00'}, {'author': None, 'title': 'Vestas Sw

In [235]:
#@title Function calling for FreshPrompt


def is_valid_article(article):
    """
    Check if an article is valid. It must be a non-empty dictionary with all required fields filled,
    and no fields should be empty arrays or empty strings where not applicable.
    """
    required_fields = ['author', 'title', 'description', 'url', 'source', 'published_at']
    # Check if the article is a non-empty dictionary
    if not isinstance(article, dict) or not article:
        return False

    # Ensure all required fields are present and non-empty (where empty means not just presence but having a meaningful value)
    for field in required_fields:
        if not article.get(field):
            return False  # Check for None or empty string

        # Additional check for fields that could incorrectly be an empty list or dict (specifically tailor this if needed)
        if isinstance(article[field], (list, dict)) and not article[field]:
            return False

    return True


def process_articles(articles):
    """ Process a list of articles, filtering out invalid ones and formatting the valid ones. """
    # Filter articles to only include valid ones
    valid_articles = [article for article in articles if is_valid_article(article)]
    # Format the filtered valid articles
    formatted_articles = [format_news_article(article) for article in valid_articles]
    return formatted_articles

def call_freshprompt_news(model, question, api_key, check_premise=False, verbose=False):
  temperature = 0.0
  max_tokens = 256
  chat_completions = True

  # Settings based on model type
  if model.startswith('gpt-4'):
      num_organic_results = 15
      num_related_questions = 3
      num_questions_and_answers = 3
      num_retrieved_evidences = 15
  else:
      num_organic_results = 15
      num_related_questions = 2
      num_questions_and_answers = 2
      num_retrieved_evidences = 5

  # Choose reasoning based on verbosity
  demo_reasonings_and_answers = verbose_demo_news_reasonings_and_answers if verbose else concise_demo_news_reasonings_and_answers

  # Generate prompts for demo examples
  demo_prompts = []
  for q, s, ra in zip(demo_news_questions, demo_search_data, concise_demo_reasonings_and_answers):
      demo_prompts.append(
          freshprompt_format_news(
              q,
              s,
              ra,
              num_organic_results,
              num_related_questions,
              num_questions_and_answers,
              num_retrieved_evidences,
          )
      )

  freshprompt_demo = ''.join(demo_prompts).strip()

  #print(freshprompt_demo)




  # Apply premise checking if required
  suffix = "\nPlease check if the question contains a valid premise before answering.\nanswer: " if check_premise else "\nanswer: "
  freshprompt_question = freshprompt_format_news(
      question,
      call_news_api(question, api_key),
      suffix,
      num_organic_results,
      num_related_questions,
      num_questions_and_answers,
      num_retrieved_evidences,
  )

  # Combine demo and actual question prompts
  fresh_prompt = freshprompt_demo + freshprompt_question

  print(freshprompt_demo)

  # Call the LLM API to generate the answer
  answer = call_llm_api(
      fresh_prompt, model, temperature, max_tokens, chat_completions
  )
  return answer

# Note: Ensure the functions `fetch_news_data`, `format_search_results`, `freshprompt_format`, and `call_llm_api` are correctly defined and imported.

#answer = call_freshprompt_news("gpt-4", "latest news miliptas ca", api_key, False, False)


In [236]:
#@title FreshPrompt


# @markdown ---
model_name = "gpt-3.5-turbo-16k" #@param ["gpt-4-0125-preview", "gpt-4-turbo-preview", "gpt-4-1106-preview", "gpt-4", "gpt-4-0613", "gpt-4-32k", "gpt-4-32k-0613", "gpt-3.5-turbo-1106", "gpt-3.5-turbo", "gpt-3.5-turbo-16k", "gpt-3.5-turbo-instruct", "gpt-3.5-turbo-0613", "gpt-3.5-turbo-16k-0613", "gpt-3.5-turbo-0301"]
check_premise = False  # @param {type:"boolean"}
# @markdown ### Ask your question here!

api_key = "9d07b5b2291388a14541b99f11f0b9a5"

question = "latest news sfo ca"  # @param {type:"string"}
answer = call_freshprompt_news(model_name, question, api_key, check_premise=False, verbose=False)
button = widgets.Button(description="SHOW ANSWER")
output = widgets.Output()


def on_button_clicked(b):
  # Display the message within the output widget.
  with output:
    print(f'\n{answer}')


button.on_click(on_button_clicked)
display(button, output)

query: What major tech merger was completed in 2024?

source: jornalespalhafato
date: 2024-05-02T06:37:34+00:00
title: Nkechi Blessing e Laide Bakare reagem quando Taiwo Hassan e Yinka Quadri finalmente se reconciliam
snippet: A postagem Nkechi Blessing e Laide Bakare reagem quando Taiwo Hassan e Yinka Quadri finalmente se reconciliam apareceu pela primeira vez no Kemi Filani News. As atrizes de Nollywood, Nkechi Blessing e Laide Bakare reagiram após a reconciliação de seus colegas mais antigos, Taiwo Hassan e Yinka Quadri. Kemi Filani relatou anos atrás que o [&#8230;]The post Nkechi Blessing e Laide Bakare reagem quando Taiwo Hassan e Yinka Quadri finalmente se reconciliam appeared first on Jornal Espalha Fato.
highlight: None

source: 4-traders
date: 2024-05-02T06:37:17+00:00
title: Vestas Swings to Loss as Lower Deliveries Hit Revenue
snippet: (marketscreener.com) By Adria Calatayud Vestas Wind Systems slid to a net loss for the first quarter and reported a fall in revenue due to l

Button(description='SHOW ANSWER', style=ButtonStyle())

Output()